#Run this before starting
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"


In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
import torch.optim as optim



In [36]:
def normal(tensor, mean, std):
    if tensor is not None:
        tensor.data.normal_(mean, std)

class FeaStConv(MessagePassing):
    def __init__(self,
                 in_channels,
                 out_channels,
                 heads=8,
                 bias=True,
                 t_inv=True,
                 **kwargs):
        super(FeaStConv, self).__init__(aggr='mean', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.t_inv = t_inv

        self.weight = Parameter(torch.Tensor(in_channels,
                                             heads * out_channels))
        self.u = Parameter(torch.Tensor(in_channels, heads))
        self.c = Parameter(torch.Tensor(heads))
        if not self.t_inv:
            self.v = Parameter(torch.Tensor(in_channels, heads))

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        normal(self.weight, mean=0, std=0.1)
        normal(self.u, mean=0, std=0.1)
        normal(self.c, mean=0, std=0.1)
        normal(self.bias, mean=0, std=0.1)
        if not self.t_inv:
            normal(self.v, mean=0, std=0.1)

    def forward(self, x, edge_index):
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        return self.propagate(edge_index, x=x, num_nodes=x.size(0))

    def message(self, x_i, x_j):
        # dim: x_i, [E, F_in];
        if self.t_inv:
            # with translation invariance
            q = torch.mm((x_i - x_j), self.u) + self.c  #[E, heads]
        else:
            q = torch.mm(x_i, self.u) + torch.mm(x_j, self.v) + self.c
        q = F.softmax(q, dim=1)  #[E, heads]

        x_j = torch.mm(x_j, self.weight).view(-1, self.heads,
                                              self.out_channels)
        return (x_j * q.view(-1, self.heads, 1)).sum(dim=1)

    def update(self, aggr_out):
        if self.bias is not None:
            aggr_out = aggr_out + self.bias
        return aggr_out

    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.heads)


In [37]:
class FeaStNet(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv=True):
        super(FeaStNet, self).__init__()

        self.fc0 = nn.Linear(in_channels, 16)
        self.conv1 = FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, num_classes)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.fc0(x))
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [54]:
class ArchNN(torch.nn.Module):
    def __init__(self, in_channels, num_classes, heads, t_inv = True):
        super(ArchNN, self).__init__()
        self.fc0 = nn.Linear(in_channels, 16)
        self.conv1 = FeaStConv(16, 32, heads=heads, t_inv=t_inv)
        self.conv2 = FeaStConv(32, 64, heads=heads, t_inv=t_inv)
        self.conv3 = FeaStConv(64, 128, heads=heads, t_inv=t_inv)
        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, num_classes)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()
        self.conv3.reset_parameters()

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.fc0(x))
        x = F.elu(self.conv1(x, edge_index))
        x = F.elu(self.conv2(x, edge_index))
        x = F.elu(self.conv3(x, edge_index))
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        print(x)
        print(F.log_softmax(x, dijm=1))
        return F.log_softmax(x, dijm=1)

    


In [49]:
#data
#TxtFile to lists
import numpy as np 

#data = np.loadtxt('C:\\Users\\eliasak\\OneDrive - NTNU\\Master thesis\\07_ML\\ArchGNN\\data.txt')

values_list = []  # create an empty list to store the values for each row
x = []
z = []
m = []
l = []

with open("data.txt", "r") as f:
    for line in f:  # loop over each line in the file
        values = line.strip().split("\t")  # split the line into a list of strings

        xL = []
        zL = []
        mL = []
        lL = []

        for i in range(20):
            xL.append(round(float(values[i]),3))
            zL.append(round(float(values[i+20]),3))
            mL.append(round(float(values[i+20*2]),3))
            try:
                lL.append(round(float(values[i+20*3]),3))
            except:
                None
        x.append(xL)
        z.append(zL)
        m.append(mL)
        l.append(lL)


dataY = []

dataX = []
for i in range(len(x)):
    dataXs = []
    for j in range(0,len(x[i])):
        dataX0 = []
        dataX0.append(j)
        dataX0.append(x[i][j])
        dataX0.append(z[i][j])
        dataXs.append(dataX0)
    dataX.append(dataXs)

dataX = np.array(dataX)
dataY = np.array(m)

dataEgdeIndex = []
for i in range(len(x[0])-1):
    dataEgdeIndex.append([i,i+1])
    dataEgdeIndex.append([i+1,i])  

dataEgdeIndex = np.array(dataEgdeIndex) 
dataEgdeIndex =np.transpose(dataEgdeIndex)


input_data = dataX
target_data = dataY
edge_index = dataEgdeIndex
dataX = torch.from_numpy(dataX)
dataY = torch.from_numpy(dataY)
edge_index = torch.from_numpy(edge_index)

dataset = []
for i in range(dataX.shape[0]):
    dataset.append(Data(x=dataX[i], edge_index=edge_index, y=dataY[i]))

train_loader = dataset[:200]
test_loader = dataset[200:]
print(dataset[0].y)
print(dataset[0].x)

tensor([ 5.5400, -1.2220,  1.5160,  2.7780,  2.8050,  1.9420,  0.5850, -0.8640,
        -2.0530, -2.7170, -2.7170, -2.0530, -0.8640,  0.5850,  1.9420,  2.8050,
         2.7780,  1.5160, -1.2220, -5.5400], dtype=torch.float64)
tensor([[  0.0000,   0.0000,   0.0000],
        [  1.0000,   1.0230,  12.3450],
        [  2.0000,   4.0640,  24.3520],
        [  3.0000,   9.0390,  35.6960],
        [  4.0000,  15.8140,  46.0660],
        [  5.0000,  24.2040,  55.1790],
        [  6.0000,  33.9790,  62.7870],
        [  7.0000,  44.8730,  68.6830],
        [  8.0000,  56.5890,  72.7050],
        [  9.0000,  68.8070,  74.7440],
        [ 10.0000,  81.1930,  74.7440],
        [ 11.0000,  93.4110,  72.7050],
        [ 12.0000, 105.1270,  68.6830],
        [ 13.0000, 116.0210,  62.7870],
        [ 14.0000, 125.7960,  55.1790],
        [ 15.0000, 134.1860,  46.0660],
        [ 16.0000, 140.9610,  35.6960],
        [ 17.0000, 145.9360,  24.3520],
        [ 18.0000, 148.9770,  12.3450],
        [ 19.0

In [52]:
import time
import torch
import torch.nn.functional as F


def print_info(info):
    message = ('Epoch: {}/{}, Duration: {:.3f}s, ACC: {:.4f}, '
               'Train Loss: {:.4f}, Test Loss:{:.4f}').format(
                   info['current_epoch'], info['epochs'], info['t_duration'],
                   info['acc'], info['train_loss'], info['test_loss'])
    print(message)


def run(model, train_loader, test_loader, num_nodes, epochs, optimizer):

    for epoch in range(1, epochs + 1):
        t = time.time()
        train_loss = train(model, train_loader, optimizer)
        t_duration = time.time() - t
        acc, test_loss = test(model, test_loader, num_nodes)
        eval_info = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'acc': acc,
            'current_epoch': epoch,
            'epochs': epochs,
            't_duration': t_duration
        }

        print_info(eval_info)


def train(model, train_loader, optimizer):
    model.train()

    total_loss = 0
    for idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        print(model(data))
        loss = F.nll_loss(model(data), data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)


def test(model, test_loader, num_nodes):
    model.eval()
    correct = 0
    total_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for idx, data in enumerate(test_loader):
            out = model(data)
            total_loss += F.nll_loss(out, data.y).item()
            pred = out.max(1)[1]
            correct += pred.eq(data.y).sum().item()
            n_graphs += data.num_graphs
    return correct / (n_graphs * num_nodes), total_loss / len(test_loader)


In [55]:
#runner
num_nodes = train_loader[0].x.shape[0]
num_features = train_loader[0].x.shape[1]

model = ArchNN(num_features, num_nodes, heads=10)

optimizer = optim.Adam(model.parameters(),
                       lr=0.01)


run(model, train_loader, test_loader, num_nodes, 10, optimizer)

RuntimeError: mat1 and mat2 must have the same dtype

In [ ]:
#OLD
# Define a simple GNN model
class MyGNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyGNN, self).__init__()
        
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x

# Define a simple training function for the GNN model
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = F.mse_loss(output, data.y)
    loss.backward()
    optimizer.step()
    return loss.item()

# Define a simple testing function for the GNN model
def test(model, data):
    model.eval()
    output = model(data.x, data.edge_index)
    return output

# Set up the input data
x = torch.tensor([
    [0,0,1,1,1,0,0,0],
    [10,5,0,0,0,0,5,0],
    [20,10,0,0,0,0,5,0],
    [30,5,0,0,0,0,5,0],
    [40,0,1,1,1,0,0,0]
], dtype=torch.float)

edge_index = torch.tensor([
    [0,1], [1,0], [1,2], [2,1], [2,3], [3,2], [3,4], [4,3]
], dtype=torch.long).t()

# Set up the output data (nodal forces and moments)
y = torch.tensor([
    [0,0,0,0,0,10,20,0],
    [0,0,0,0,0,5,0,0],
    [0,0,0,0,0,5,0,0],
    [0,0,0,0,0,5,0,0],
    [0,0,0,0,0,10,20,0]
], dtype=torch.float)

# Create a Data object that encapsulates the input and output data
data = Data(x=x, edge_index=edge_index, y=y)

# Initialize the GNN model
input_dim = x.shape[1]
output_dim = y.shape[1]
hidden_dim = 16
model = MyGNN(input_dim, hidden_dim, output_dim)

# Define the optimizer and the number of epochs for training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_epochs = 100

# Train the GNN model
for epoch in range(num_epochs):
    loss = train(model, data, optimizer)
    print('Epoch {}: loss={}'.format(epoch, loss))

# Test the GNN model on a new input example
x_new = torch.tensor([
    [0,0,1,1,1,0,0,0],
    [10,5,0,0,0,0,5,0],
    [20,10,0,0,0,0,5,0],
    [30,5,0,0,0,0,5,0]
])


In [4]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121Note: you may need to restart the kernel to use updated packages.



In [1]:
import torch
print(torch.cuda.is_available())

False


In [4]:
pip install numba

  Using cached numba-0.56.4.tar.gz (2.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\kt\AppData\Local\Temp\pip-install-zrcxyxi6\numba_7683b5d5502042c187dcc44c323381b0\setup.py", line 51, in <module>
          _guard_py_ver()
        File "C:\Users\kt\AppData\Local\Temp\pip-install-zrcxyxi6\numba_7683b5d5502042c187dcc44c323381b0\setup.py", line 48, in _guard_py_ver
          raise RuntimeError(msg.format(cur_py, min_py, max_py))
      RuntimeError: Cannot install on Python version 3.11.0; only versions >=3.7,<3.11 are supported.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is

In [1]:
from numba import jit, cuda

ModuleNotFoundError: No module named 'numba'